In [2]:
import os
import sys
sys.path.append('../../')
import backtrader as bt
import akshare as ak
import pandas as pd
from datetime import datetime


/Users/alsc/VscodeProject/hh_quant/.venv/lib/python3.8/site-packages/akshare/__init__.py:2683: UserWarning: 为了支持更多特性，请将 Python 升级到 3.9.0 及以上版本！
  warnings.warn(


In [3]:
data_path = '../backtest_data/stock_data/600519.pkl'
# 读取原始数据
df = pd.read_pickle(data_path).iloc[:, :6]
# 设置数据index
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index("datetime", inplace=True)
# 配置日期格式
fmt_start_date = datetime.strptime('20100101', '%Y%m%d')
fmt_end_date = datetime.strptime('20150101', '%Y%m%d')
# 过滤期间数据
df = df[(df.index >= fmt_start_date) & (df.index <= fmt_end_date)]

# data_hs300 = bt.feeds.PandasData(dataname=hs300_df)
# data_600519 = bt.feeds.PandasData(dataname=stock_df)

df.head()

,open,close,high,low,volume
datetime,,,,,
2010-01-04,172.00,169.94,172.00,169.31,44305
2010-01-05,170.99,169.44,171.50,169.00,31513
2010-01-06,168.99,166.76,169.50,166.31,39889
2010-01-07,166.76,163.72,167.19,161.88,48826
2010-01-08,164.00,162.00,164.00,160.10,36702


In [4]:
# df.info()

In [5]:
# 获取沪深300指数历史数据
hs300_df = ak.stock_zh_index_daily(symbol="sh000300").rename(columns={
    'date': 'datetime'
})

# 获取贵州茅台股票历史数据
stock_df = ak.stock_zh_a_daily(symbol="sh600519", adjust="").iloc[:, :6].rename(columns={
    'date': 'datetime'
})

# 设置时间索引
hs300_df['datetime'] = pd.to_datetime(hs300_df['datetime'])
stock_df['datetime'] = pd.to_datetime(stock_df['datetime'])
hs300_df.set_index('datetime', inplace=True)
stock_df.set_index('datetime', inplace=True)

# 设置开始和结束日期
start_date = "2010-01-01"
end_date = "2015-01-01"

# 过滤指定时间段的数据
hs300_df = hs300_df[(hs300_df.index >= start_date) & (hs300_df.index <= end_date)]
stock_df = stock_df[(stock_df.index >= start_date) & (stock_df.index <= end_date)]

data_hs300 = bt.feeds.PandasData(dataname=hs300_df)
data_600519 =  bt.feeds.PandasData(dataname=stock_df)

In [6]:
hs300_df.head()
# hs300_df.info()

,open,high,low,close,volume
datetime,,,,,
2010-01-04,3592.468,3597.748,3535.229,3535.229,6610108000
2010-01-05,3545.186,3577.526,3497.661,3564.038,8580964000
2010-01-06,3558.700,3588.832,3541.173,3541.727,7847312500
2010-01-07,3543.160,3558.556,3452.769,3471.456,8035004000
2010-01-08,3456.908,3482.083,3426.698,3480.130,6079025200


In [ ]:
stock_df.head()

In [ ]:
class MovingAverageStrategy(bt.Strategy):
    params = (('maperiod', 15),)

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.maperiod)

    def next(self):
        if not self.position:
            if self.data.close > self.sma:
                self.order = self.buy()
        elif self.position:
            if self.data.close < self.sma:
                self.order = self.sell()

In [ ]:
# 设置回测基础参数
print("设置回测基础参数...")
BACKTEST_INITIAL_CASH = 100000  # 初始化资金
# BACKTEST_COMMISSION = 0.0003  # 初始化双边佣金0.0003
BACKTEST_SIZER = 100  # 设定每笔交易100股
# BACKTEST_SLIPPAGE_TYPE = "perc"  # 初始化双边滑点类型
# BACKTEST_SLIPPAGE_VALUE = 0.0001  # 初始化双边滑点0.0001
# BACKTEST_START_DATE = "20200101"  # 回测开始日期
# BACKTEST_END_DATE = "20240101"  # 回测结束日期
# BACKTEST_STOCK_SYMBOLS = ["300197", "300810", "300125"]  # 回测股票代码
# BACKTEST_OPTIMIZE_STRATEGY = False  # 是否启用优化策略

# 初始化Cerebro
print("开始初始化Cerebro...")
cerebro = bt.Cerebro()
cerebro.broker.setcash(BACKTEST_INITIAL_CASH)
# cerebro.broker.setcommission(commission=BACKTEST_COMMISSION)
# if BACKTEST_SLIPPAGE_TYPE == "fix":  # 每笔交易滑点为固定值
#     cerebro.broker.set_slippage_fixed(BACKTEST_SLIPPAGE_VALUE)
# elif BACKTEST_SLIPPAGE_TYPE == "perc":  # 每笔交易滑点为百分比
#     cerebro.broker.set_slippage_perc(BACKTEST_SLIPPAGE_VALUE)
# cerebro.addsizer(bt.sizers.FixedSize, stake=BACKTEST_SIZER)

# 添加分析器
print("开始添加分析器...")
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="pnl")  # 盈亏分析器
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annual_return")  # 年化收益分析器
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio")  # 普比率分析器
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")  # 最大回撤分析器

# 添加MovingAverageStrategy
print("开始添加策略...")
cerebro.addstrategy(MovingAverageStrategy)

# 导入对应的股票数据到cerebro
print("开始导入数据...")
# cerebro.adddata(data_hs300, name='hs300')
cerebro.adddata(data_600519, name='600519')

# 设置基准观察者，追踪沪深300指数的表现
# cerebro.addobserver(bt.observers.Benchmark,
#                     data=data_hs300,
#                     timeframe=bt.TimeFrame.NoTimeFrame)

# 运行回测
print("开始运行回测...")
results = cerebro.run()

# 结束回测
print("结束回测...")
print(f"Start Portfolio Value: {BACKTEST_INITIAL_CASH:.2f}")
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

In [ ]:
# strat = results[0]
# pyfoliozer = strat.analyzers.getbyname('pyfolio')
# returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

In [ ]:
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo

bo = Bokeh(style='bar', plot_mode='single', scheme=Tradimo())
cerebro.plot(bo)

In [7]:
a = [1,2,3,4,5]

In [8]:
a[:-1]

[1, 2, 3, 4]

In [9]:
a = {
    'a': 1,
    'b': 2
}

In [11]:
for i, j in a.items():
    print(j)

1
2


In [2]:
a = 1

if a:
    print('aaa')

aaa


In [1]:
a = [1,2,3,4,5]
a.remove(2)